# Matrix Addition Using the Triton Kernel

No CUDA:
- Você pensa em threads individuais.

> "Cada thread soma um número."

No Triton:
- Você pensa em blocos vetorizados.
- Você opera em vetores.
- O compilador gera o paralelismo.
- É uma abstração mais moderna.

> "Esse bloco soma um vetor de números."

In [3]:
import torch
import triton
import triton.language as tl

In [19]:
@triton.jit
def addition_fn(A_pointer, B_pointer, C_pointer, n_elements, BLOCK_SIZE: tl.constexpr):
    # é o nosso BLOCO
    pid = tl.program_id(0)

    # Cada pid pega um bloco diferente da memória. Isso é como o threadsPerBlock
    # [0, 1, 2, ..., 15]
    offsets = pid * BLOCK_SIZE + tl.arange(0, BLOCK_SIZE)
    # Se a matriz fosse maior que o bloco, isso impediria erro.
    mask = offsets < n_elements

    # CPU → GPU
    A = tl.load(A_pointer + offsets, mask=mask)
    B = tl.load(B_pointer + offsets, mask=mask)
    # Add
    C = A + B

    tl.store(C_pointer + offsets, C, mask=mask)

In [5]:
A = torch.ones((4, 4), device="cuda", dtype=torch.float32)
B = torch.full((4, 4), 2.0, device="cuda", dtype=torch.float32)

In [9]:
print(A)
print()
print(B)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], device='cuda:0')

tensor([[2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.]], device='cuda:0')


In [11]:
C = torch.empty_like(A)

In [13]:
n_elements = A.numel()

In [15]:
BLOCK_SIZE = 16

In [20]:
grid = lambda meta: (triton.cdiv(n_elements, meta["BLOCK_SIZE"]),)

In [21]:
addition_fn[grid](A, B, C, n_elements, BLOCK_SIZE=BLOCK_SIZE)

In [23]:
print("A:")
print(A)

print("B:")
print(B)

print('-' * 30)
print("C:")
print(C)

A:
tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], device='cuda:0')
B:
tensor([[2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.],
        [2., 2., 2., 2.]], device='cuda:0')
------------------------------
C:
tensor([[3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.],
        [3., 3., 3., 3.]], device='cuda:0')
